In [1]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import numpy as np

# Load the dataset
data_dir = "./processed"
img_height, img_width = 224, 224  # specify the height and width to resize images
batch_size = 32

dataset = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size
)

Found 21600 files belonging to 27 classes.


In [2]:
# Get the class names and convert them to numerical labels
class_names = dataset.class_names
encoder = LabelEncoder()
labels = encoder.fit_transform(class_names)

In [3]:
ls = []

print("Length of dataset: ", len(dataset))
print("Length of labels: ", len(labels))

Length of dataset:  675
Length of labels:  27


In [10]:
# Split the dataset into training and testing sets
# x_train, x_test, y_train, y_test = train_test_split(dataset, labels, test_size=0.2, random_state=16)

# x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)


In [14]:
# convert the dataset type into array 
print(dataset)

<_BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [15]:
import numpy as np

# Initialize empty lists to hold the images and labels
images = []
labels_list = []

# Iterate over the dataset
for image_batch, label_batch in dataset:
    # Convert the batch to numpy arrays and add them to the lists
    images.append(image_batch.numpy())
    labels_list.append(label_batch.numpy())

# Concatenate the batches
images = np.concatenate(images)
labels_list = np.concatenate(labels_list)

# Flatten the images
images = images.reshape(images.shape[0], -1)

# Split the dataset into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(images, labels_list, test_size=0.2, random_state=42)

: 

In [ ]:
# Train the Random Forest model
model = RandomForestClassifier(n_estimators=100)
model.fit(x_train, y_train)

In [ ]:
# save the model to .keras file
model.save("model.h5")

In [ ]:
# Evaluate the model
y_pred = model.predict(x_test)
print("Accuracy: ", accuracy_score(y_test, y_pred))